In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPEN_AI_KEY"] = api_key

In [ ]:
### OUTER ANALYZER GENERATION WITH RAG

import os
import random
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Step 1: Load and chunk the codebase, then create embeddings
def index_codebase(directory):
    # Load all .py files from the directory
    loader = DirectoryLoader(
        path=directory,
        glob="**/*.py",  # Only .py files
        exclude=["**/__pycache__/**", "**/*.pyc"]
    )
    documents = loader.load()
    
    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        separators=['\n\n', '\n', ' ', '']
    )
    chunks = text_splitter.split_documents(documents)
    
    # Create embeddings for each chunk
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(chunks, embeddings)
    
    return vectorstore

# Initialize the vector store
codebase_directory = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\mobile_insight'
vectorstore = index_codebase(codebase_directory)

# Step 2: Load n examples for in-context learning from the folder
def load_in_context_examples(example_folder, n=3):
    examples = []
    
    # Get lists of .txt and .py files
    txt_files = [f for f in os.listdir(example_folder) if f.startswith("prompt_") and f.endswith('.txt')]
    py_files = [f for f in os.listdir(example_folder) if f.startswith("modified_") and f.endswith('.py')]
    
    # Create a dictionary for .py files based on the base filename
    py_file_dict = {f.replace("modified_", "").replace(".py", ""): f for f in py_files}
    
    for txt_file in txt_files:
        # Get the base name by removing "prompt_" and ".txt"
        base_name = txt_file.replace("prompt_", "").replace(".txt", "")
        
        # Check if there is a corresponding .py file
        if base_name in py_file_dict:
            txt_file_path = os.path.join(example_folder, txt_file)
            py_file_path = os.path.join(example_folder, py_file_dict[base_name])
            
            # Load the prompt + inner analyzer code from the .txt file
            with open(txt_file_path, 'r') as f:
                prompt_inner_code = f.read().strip()
            
            # Load the expected outer analyzer code from the .py file
            with open(py_file_path, 'r') as f:
                expected_outer_code = f.read().strip()
            
            examples.append({
                'prompt_inner_code': prompt_inner_code,
                'expected_outer_code': expected_outer_code
            })
    
    # Randomly sample n examples
    return random.sample(examples, min(n, len(examples)))

# Step 3: Define the prompt template for generating outer analyzers with RAG context
def create_prompt_template(n):
    in_context_placeholders = "\n\n".join(
        [f"Example {i+1}:\nPrompt + Inner Analyzer Code:\n{{example{i+1}_prompt_inner_code}}\nExpected Outer Analyzer Code:\n{{example{i+1}_expected_outer_code}}"
         for i in range(n)]
    )
    
    return PromptTemplate(
        input_variables=[f"example{i+1}_prompt_inner_code" for i in range(n)] +
                        [f"example{i+1}_expected_outer_code" for i in range(n)] +
                        ["retrieved_context", "target_prompt_inner_code"],
        template=f"""

You are an AI assistant that generates code for outer analyzers using the Mobileinsight-core Python library, a library that enables below-IP, \
fine-grained mobile network analytics on end devices. It is a cross-platform package for mobile network monitoring and analysis.

Below are {n} examples of prompt + inner analyzer code pairs along with their corresponding expected outer analyzer code.

{in_context_placeholders}

Additionally, here is some relevant context from the Mobileinsight codebase:

{{retrieved_context}}

Given the following prompt and inner analyzer code, generate the corresponding outer analyzer:

{{target_prompt_inner_code}}

NOTE: PLEASE PROVIDE ONLY THE CODE AND NOTHING ELSE, AS THIS OUTPUT IS BEING DIRECTLY SAVED TO A .PY FILE AND RAN AUTONOMOUSLY. \
ADDITIONALLY, ENSURE THAT YOU PROVIDE THE FULL COMPLETE CODE, AND DO NOT LEAVE OUT ANY PARTS FOR THE USER TO COMPLETE. THE CODE SHOULD FULLY RUN \
WITH NO ADDITIONAL MODIFICATIONS REQUIRED.

Generated Outer Analyzer:
"""
    )

# Step 4: Initialize the LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

# Step 5: Function to retrieve relevant documents from the vector store
def retrieve_relevant_documents(prompt_inner_code):
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    relevant_docs = retriever.get_relevant_documents(prompt_inner_code)
    retrieved_context = "\n\n".join([doc.page_content for doc in relevant_docs])
    return retrieved_context

# Step 6: Function to generate outer analyzer based on in-context examples and retrieved context
def generate_outer_analyzer(example_folder, target_txt_file, n=3, output_folder="generated_outer_analyzers"):
    # Load in-context examples
    examples = load_in_context_examples(example_folder, n)
    
    # Load the target prompt + inner analyzer code from the target .txt file
    with open(target_txt_file, 'r') as f:
        target_prompt_inner_code = f.read().strip()
    
    # Retrieve relevant context from the vector store
    retrieved_context = retrieve_relevant_documents(target_prompt_inner_code)
    
    # Prepare the prompt template
    prompt_template = create_prompt_template(n)
    chain = LLMChain(llm=llm, prompt=prompt_template)

    # Prepare inputs with in-context examples, retrieved context, and the target example
    inputs = {f"example{i+1}_prompt_inner_code": examples[i]['prompt_inner_code'] for i in range(n)}
    inputs.update({f"example{i+1}_expected_outer_code": examples[i]['expected_outer_code'] for i in range(n)})
    inputs["retrieved_context"] = retrieved_context
    inputs["target_prompt_inner_code"] = target_prompt_inner_code

    # Format the prompt with the inputs to display the populated prompt
    # formatted_prompt = prompt_template.format(**inputs)
    # print("Formatted Prompt:\n")
    # print(formatted_prompt)  # Print the fully populated prompt for inspection

    # Generate the outer analyzer code
    result = chain.invoke(inputs)
    generated_outer_code = result['text'].strip()
    code = generated_outer_code.replace("```python", "", 1)
    code = code.replace("```", "", 1)

    # Save the generated code to a .py file
    os.makedirs(output_folder, exist_ok=True)
    output_filename = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(target_txt_file))[0]}_generated_RAG.py")
    with open(output_filename, 'w') as f:
        f.write(code)
    
    print(f"Generated outer analyzer saved to {output_filename}")

# # Example usage
# example_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_working_dataset'
# target_txt_file = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_working_dataset\prompt_lte_phy_analyzer_example_1.txt'
# output_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_analyzers'

# # Generate the outer analyzer
# generate_outer_analyzer(example_folder, target_txt_file, n=3, output_folder=output_folder)


C:\Users\bhull\AppData\Local\Temp\ipykernel_6452\2459149841.py:30: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [3]:
### LOOP THROUGH OUTER ANALYZERS

def process_all_txt_files(example_folder, output_folder, n=3):
    # Get all `.txt` files that start with "prompt_"
    txt_files = [f for f in os.listdir(example_folder) if f.startswith("prompt_") and f.endswith('.txt')]

    # Iterate through each `.txt` file and call the generate_inner_analyzer function
    for txt_file in txt_files:
        target_txt_file = os.path.join(example_folder, txt_file)
        print(f"Processing {target_txt_file}...")

        # Call the generate_inner_analyzer function
        try:
            generate_outer_analyzer(example_folder, target_txt_file, n=n, output_folder=output_folder)
        except Exception as e:
            print(f"Error processing {txt_file}: {e}")

# Define the input folder containing .txt and .py files
example_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_working_dataset'

# Define the output folder where the generated inner analyzers will be saved
output_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_analyzers_RAG'

# Process all .txt files
process_all_txt_files(example_folder, output_folder, n=3)

Processing C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_working_dataset\prompt_dl_retx_analyzer_test_1.txt...


C:\Users\bhull\AppData\Local\Temp\ipykernel_6452\2459149841.py:117: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(prompt_inner_code)
C:\Users\bhull\AppData\Local\Temp\ipykernel_6452\2459149841.py:135: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


Generated outer analyzer saved to C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_analyzers_RAG\prompt_dl_retx_analyzer_test_1_generated_RAG.py
Processing C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_working_dataset\prompt_dl_retx_analyzer_test_2.txt...
Generated outer analyzer saved to C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_analyzers_RAG\prompt_dl_retx_analyzer_test_2_generated_RAG.py
Processing C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_working_dataset\prompt_dl_retx_analyzer_test_3.txt...
Generated outer analyzer saved to C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_analyzers_RAG\prompt_dl_retx_analyzer_t

In [4]:
import os
import random
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Step 1: Load and chunk the codebase, then create embeddings
def index_codebase(directory):
    # Load all .py files from the directory
    loader = DirectoryLoader(
        path=directory,
        glob="**/*.py",  # Only .py files
        exclude=["**/__pycache__/**", "**/*.pyc"]
    )
    documents = loader.load()
    
    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        separators=['\n\n', '\n', ' ', '']
    )
    chunks = text_splitter.split_documents(documents)
    
    # Create embeddings for each chunk
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(chunks, embeddings)
    
    return vectorstore

# Initialize the vector store
codebase_directory = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\mobile_insight'
vectorstore = index_codebase(codebase_directory)

# Step 2: Load n examples for in-context learning from the folder
def load_in_context_examples(example_folder, n=3):
    examples = []
    
    # Get lists of .txt and .py files
    txt_files = [f for f in os.listdir(example_folder) if f.startswith("prompt_") and f.endswith('.txt')]
    py_files = [f for f in os.listdir(example_folder) if f.startswith("modified_") and f.endswith('.py')]
    
    # Create a dictionary for .py files based on the base filename
    py_file_dict = {f.replace("modified_", "").replace(".py", ""): f for f in py_files}
    
    for txt_file in txt_files:
        # Get the base name by removing "prompt_" and ".txt"
        base_name = txt_file.replace("prompt_", "").replace(".txt", "")
        
        # Check if there is a corresponding .py file
        if base_name in py_file_dict:
            txt_file_path = os.path.join(example_folder, txt_file)
            py_file_path = os.path.join(example_folder, py_file_dict[base_name])
            
            # Load the prompt + outer analyzer code from the .txt file
            with open(txt_file_path, 'r') as f:
                prompt_outer_code = f.read().strip()
            
            # Load the expected inner analyzer code from the .py file
            with open(py_file_path, 'r') as f:
                expected_inner_code = f.read().strip()
            
            examples.append({
                'prompt_outer_code': prompt_outer_code,
                'expected_inner_code': expected_inner_code
            })
    
    # Randomly sample n examples
    return random.sample(examples, min(n, len(examples)))

# Step 3: Define the prompt template for generating inner analyzers with RAG context
def create_prompt_template(n):
    in_context_placeholders = "\n\n".join(
        [f"Example {i+1}:\nPrompt + Outer Analyzer Code:\n{{example{i+1}_prompt_outer_code}}\nExpected Inner Analyzer Code:\n{{example{i+1}_expected_inner_code}}"
         for i in range(n)]
    )
    
    return PromptTemplate(
        input_variables=[f"example{i+1}_prompt_outer_code" for i in range(n)] +
                        [f"example{i+1}_expected_inner_code" for i in range(n)] +
                        ["retrieved_context", "target_prompt_outer_code"],
        template=f"""

You are an AI assistant that generates code for inner analyzers using the Mobileinsight-core Python library, a library that enables below-IP, \
fine-grained mobile network analytics on end devices. It is a cross-platform package for mobile network monitoring and analysis.

Below are {n} examples of prompt + outer analyzer code pairs along with their corresponding expected inner analyzer code.

{in_context_placeholders}

Additionally, here is some relevant context from the Mobileinsight codebase:

{{retrieved_context}}

Given the following prompt and outer analyzer code, generate the corresponding inner analyzer:

{{target_prompt_outer_code}}

NOTE: PLEASE PROVIDE ONLY THE CODE AND NOTHING ELSE, AS THIS OUTPUT IS BEING DIRECTLY SAVED TO A .PY FILE AND RAN AUTONOMOUSLY. \
ADDITIONALLY, ENSURE THAT YOU PROVIDE THE FULL COMPLETE CODE, AND DO NOT LEAVE OUT ANY PARTS FOR THE USER TO COMPLETE. THE CODE SHOULD FULLY RUN \
WITH NO ADDITIONAL MODIFICATIONS REQUIRED.

Generated Inner Analyzer:
"""
    )

# Step 4: Initialize the LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

# Step 5: Function to retrieve relevant documents from the vector store
def retrieve_relevant_documents(prompt_outer_code):
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    relevant_docs = retriever.get_relevant_documents(prompt_outer_code)
    retrieved_context = "\n\n".join([doc.page_content for doc in relevant_docs])
    return retrieved_context

# Step 6: Function to generate inner analyzer based on in-context examples and retrieved context
def generate_inner_analyzer(example_folder, target_txt_file, n=3, output_folder="generated_inner_analyzers"):
    # Load in-context examples
    examples = load_in_context_examples(example_folder, n)
    
    # Load the target prompt + outer analyzer code from the target .txt file
    with open(target_txt_file, 'r') as f:
        target_prompt_outer_code = f.read().strip()
    
    # Retrieve relevant context from the vector store
    retrieved_context = retrieve_relevant_documents(target_prompt_outer_code)
    
    # Prepare the prompt template
    prompt_template = create_prompt_template(n)
    chain = LLMChain(llm=llm, prompt=prompt_template)

    # Prepare inputs with in-context examples, retrieved context, and the target example
    inputs = {f"example{i+1}_prompt_outer_code": examples[i]['prompt_outer_code'] for i in range(n)}
    inputs.update({f"example{i+1}_expected_inner_code": examples[i]['expected_inner_code'] for i in range(n)})
    inputs["retrieved_context"] = retrieved_context
    inputs["target_prompt_outer_code"] = target_prompt_outer_code

    # Format the prompt with the inputs to display the populated prompt
    formatted_prompt = prompt_template.format(**inputs)
    print("Formatted Prompt:\n")
    print(formatted_prompt)  # Print the fully populated prompt for inspection

    # Generate the inner analyzer code
    result = chain.invoke(inputs)
    generated_inner_code = result['text'].strip()
    code = generated_inner_code.replace("```python", "", 1)
    code = code.replace("```", "", 1)

    # Save the generated code to a .py file
    os.makedirs(output_folder, exist_ok=True)
    output_filename = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(target_txt_file))[0]}_generated_RAG.py")
    with open(output_filename, 'w') as f:
        f.write(code)
    
    print(f"Generated inner analyzer saved to {output_filename}")

# # Example usage
# example_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_working_dataset'
# target_txt_file = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_working_dataset\prompt_lte_phy_analyzer_1.txt'
# output_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_analyzers'

# # Generate the inner analyzer
# generate_inner_analyzer(example_folder, target_txt_file, n=3, output_folder=output_folder)


In [5]:
### LOOP THROUGH INNER ANALYZERS

def process_all_txt_files(example_folder, output_folder, n=3):
    # Get all `.txt` files that start with "prompt_"
    txt_files = [f for f in os.listdir(example_folder) if f.startswith("prompt_") and f.endswith('.txt')]

    # Iterate through each `.txt` file and call the generate_inner_analyzer function
    for txt_file in txt_files:
        target_txt_file = os.path.join(example_folder, txt_file)
        print(f"Processing {target_txt_file}...")

        try:
            generate_inner_analyzer(example_folder, target_txt_file, n=n, output_folder=output_folder)
        except Exception as e:
            print(f"Error processing {txt_file}: {e}")

# Define the input folder containing .txt and .py files
example_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_working_dataset'

# Define the output folder where the generated inner analyzers will be saved
output_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_analyzers_RAG'

# Process all .txt files
process_all_txt_files(example_folder, output_folder, n=3)

Processing C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_working_dataset\prompt_kpi_manager_1.txt...
Formatted Prompt:



You are an AI assistant that generates code for inner analyzers using the Mobileinsight-core Python library, a library that enables below-IP, fine-grained mobile network analytics on end devices. It is a cross-platform package for mobile network monitoring and analysis.

Below are 3 examples of prompt + outer analyzer code pairs along with their corresponding expected inner analyzer code.

Example 1:
Prompt + Outer Analyzer Code:
Prompt: I want you to define a class `ModifiedWcdmaRrcAnalyzer` that inherits from a base `ProtocolAnalyzer` class, and returns modified metrics for WCDMA RRC analysis:

1. Class Definition: `ModifiedWcdmaRrcAnalyzer`
This class extends from `ProtocolAnalyzer` and is designed to analyze WCDMA (3G) Radio Resource Control (RRC) protocol with adjusted metrics. It should